In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import plot_importance, plot_tree

DATA_PATH = "../input/ashrae-energy-prediction/"
LEAKDATA_PATH = "../input/leakdata/"



In [2]:
train = pd.read_pickle("train.pkl", compression = 'zip')
test = pd.read_pickle("test.pkl", compression = 'zip')

In [3]:
remove_features = ['year','timestamp','meter_reading', "precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]
features = []
for col in train.columns:
    if col not in remove_features:
        features.append(col)
#for col in ['building_id', 'site_id','meter','primary_use','square_feet','year_built','floor_count']:
#    try:features.remove(col)
#    except:None

In [4]:
categorical_feats = []

## Train

In [5]:
num_folds = 3
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for site_id in train['site_id'].unique():
    a += 1
    print(a)
    for meter in train[train['site_id'] == site_id]['meter'].unique():
        train_small = train[(train['site_id'] == site_id) & (train['meter'] == meter)]
        test_small = test[(test['site_id'] == site_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = xgb.XGBRegressor(
                      n_estimators=6000,
                      max_depth=12,
                      num_boost_round=500,
                      learning_rate=0.05,
                      subsample=0.8,
                      colsample_bytree=0.4,
                      missing=np.nan,
                      objective ='reg:squarederror',
                      tree_method='hist'
                      )
            h = clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading'] = predictions / num_folds
        test_small['meter_reading'] = test_small['meter_reading'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

1
[0]	validation_0-rmse:4.0671
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[276]	validation_0-rmse:0.364718

[0]	validation_0-rmse:3.98486
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[146]	validation_0-rmse:0.453245

[0]	validation_0-rmse:3.95473
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[374]	validation_0-rmse:0.643221

[0]	validation_0-rmse:3.7729
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[181]	validation_0-rmse:1.51138

[0]	validation_0-rmse:2.58977
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[62]	validation_0-rmse:1.61103

[0]	validation_0-rmse:2.91297
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[37]	validation_0-rmse:1.78827

2
[0]	validation_0-rmse:3.81435
Will train until validation_0-rmse ha

MemoryError: 

In [ ]:
num_folds = 4
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for building_id in train['building_id'].unique():
    a += 1
    print(a)
    for meter in train[train['building_id'] == building_id]['meter'].unique():
        train_small = train[(train['building_id'] == building_id) & (train['meter'] == meter)]
        test_small = test[(test['building_id'] == building_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = xgb.XGBRegressor(
                      n_estimators=6000,
                      max_depth=12,
                      num_boost_round=500,
                      learning_rate=0.1,
                      subsample=0.8,
                      colsample_bytree=0.4,
                      missing=np.nan,
                      objective ='reg:squarederror',
                      tree_method='hist'
                      )
            h = clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=50,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_building_id'] = predictions / num_folds
        test_small['meter_reading_building_id'] = test_small['meter_reading_building_id'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_building_id']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_building_id']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

In [ ]:
num_folds = 4
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for primary_use in train['primary_use'].unique():
    a += 1
    print(a, primary_use)
    for meter in train[train['primary_use'] == primary_use]['meter'].unique():
        train_small = train[(train['primary_use'] == primary_use) & (train['meter'] == meter)]
        test_small = test[(test['primary_use'] == primary_use) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = xgb.XGBRegressor(
                      n_estimators=6000,
                      max_depth=12,
                      num_boost_round=500,
                      learning_rate=0.05,
                      subsample=0.8,
                      colsample_bytree=0.4,
                      missing=np.nan,
                      objective ='reg:squarederror',
                      tree_method='hist'
                      )
            #h = clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_primary_use'] = predictions / num_folds
        test_small['meter_reading_primary_use'] = test_small['meter_reading_primary_use'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_primary_use']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_primary_use']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

In [ ]:
sub = pd.read_csv("sample_submission.csv")
sub["meter_reading"] = test['meter_reading'] * 0.6 + test['meter_reading_primary_use'] * 0.2 + test['meter_reading_building_id'] * 0.2
#sub.to_csv("best_xgb.gz",  index = False, compression= 'gzip')

In [ ]:
#test = test.rename(columns={'meter_reading':'meter_reading_site_id'})
#test["meter_reading"] = test['meter_reading_site_id'] #* 0.6 + test['meter_reading_primary_use'] * 0.2 + test['meter_reading_building_id'] * 0.2

In [ ]:
test['meter_reading'] = sub["meter_reading"]

In [ ]:
test['meter_reading'] = [meter_r * 0.85 if site not in [0,1,2,3,4,15] else meter_r for meter_r,site in zip(test['meter_reading'], test['site_id'])]

In [ ]:
leak_df = pd.read_csv(LEAKDATA_PATH+'leak.csv')
leak_df['timestamp'] = pd.to_datetime(leak_df["timestamp"])
leak_df.dropna(0, inplace=True)
leak_df = leak_df[(leak_df.timestamp.dt.year > 2016) & (leak_df.timestamp.dt.year < 2019)]
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0 # remove large negative values
leak_df = leak_df[leak_df.building_id!=245]
leak_df['meter_reading_sc'] = leak_df['meter_reading'] 
del leak_df['meter_reading'] 
test = test.merge(leak_df, on=['building_id','meter','timestamp'], how='left')

In [ ]:
test['meter_reading'] = [a if (a >= 0 and site != 4) or (a < 0 and site != 4) else b for a,b,site in zip(test['meter_reading_sc'], test['meter_reading'], test['site_id'])]

In [ ]:
sub["meter_reading"] = test['meter_reading']
sub.to_csv("../sub/xgb_3wise.csv",  index = False,float_format="%.4f")